In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graph

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-23 21:10:10--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.73MB/s    in 0.2s    

2022-01-23 21:10:10 (5.73 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"

spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [7]:
vine_df=vine_df.withColumn('total_votes',vine_df['total_votes'].cast("float").alias('total_votes'))

In [8]:
high_votes=vine_df.filter('total_votes>=20')
high_votes.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|       31.0|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|       25.0|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|       31.0|   N|                Y|
| RVTYWID2TPMMY|          2|           35|       42.0|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|       28.0|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|       64.0|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|       43.0|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|       20.0|   N|                Y|
|  R39GSNW76GYF|          1|           20|       23.0|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|       36.0|   N|     

In [9]:
ok_votes=high_votes.filter('helpful_votes/total_votes>=0.5')
ok_votes.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|       31.0|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|       25.0|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|       31.0|   N|                Y|
| RVTYWID2TPMMY|          2|           35|       42.0|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|       28.0|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|       64.0|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|       43.0|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|       20.0|   N|                Y|
|  R39GSNW76GYF|          1|           20|       23.0|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|       36.0|   N|     

In [10]:
vine_rev=ok_votes.filter('vine="Y"')
vine_rev.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3A71VR1JZD8WF|          2|           27|       30.0|   Y|                N|
|R16OMUJIGI18JZ|          5|           72|       72.0|   Y|                N|
|R3TS8ZP2FHQ9XR|          5|           39|       42.0|   Y|                N|
|R2MHP919VZN7DI|          5|           29|       30.0|   Y|                N|
| RD2BCTVS59A5L|          2|           20|       20.0|   Y|                N|
|R1JUJLXQ2CMWKF|          4|           25|       26.0|   Y|                N|
|R2T7YE0IFI6N9L|          3|           24|       25.0|   Y|                N|
|R25FI3J7WWOYXM|          3|           55|       58.0|   Y|                N|
|R3P3U4EOWIUD50|          3|           27|       36.0|   Y|                N|
|R3C7MKX7MO6XUC|          5|          216|      219.0|   Y|     

In [11]:
vine_star_reviews= vine_rev.select(['star_rating'])
vine_star_reviews.show(10)

+-----------+
|star_rating|
+-----------+
|          2|
|          5|
|          5|
|          5|
|          2|
|          4|
|          3|
|          3|
|          3|
|          5|
+-----------+
only showing top 10 rows



In [12]:
total_vine_reviews=vine_star_reviews.count()
total_vine_reviews

170

In [13]:
vine_five_star=vine_star_reviews.filter('star_rating=5').count()
vine_five_star

65

In [15]:
vine_excellent_reviews= vine_five_star/total_vine_reviews
vine_excellent_reviews

0.38235294117647056

In [16]:
non_vine_rev=ok_votes.filter('vine="N"')
non_vine_rev.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|       31.0|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|       25.0|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|       31.0|   N|                Y|
| RVTYWID2TPMMY|          2|           35|       42.0|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|       28.0|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|       64.0|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|       43.0|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|       20.0|   N|                Y|
|  R39GSNW76GYF|          1|           20|       23.0|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|       36.0|   N|     

In [ ]:
non_vine_star_reviews= non_vine_rev.select(['star_rating'])
non_vine_star_reviews.show(10)

In [18]:
total_non_vine_reviews=non_vine_star_reviews.count()
total_non_vine_reviews

37840

In [19]:
non_vine_five_star=non_vine_star_reviews.filter('star_rating=5').count()
non_vine_five_star

20612

In [20]:
non_vine_excellent_reviews= non_vine_five_star/total_non_vine_reviews
non_vine_excellent_reviews

0.5447145877378435